<a href="https://colab.research.google.com/github/artemisIsNotBeam/mewing-Ai/blob/main/mewing_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install ultralytics
!pip install tensorflow
!pip install torch
!pip install torchvision
!pip install numpy

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import tensorflow as tf
import torch
from ultralytics import YOLO

#model = torch.load('/content/drive/MyDrive/mewingAI/best.pt')
model = YOLO('/content/drive/MyDrive/mewingAI/best.pt')
#model.load('/content/drive/MyDrive/mewingAI/best.pt')

Prediction
https://docs.ultralytics.com/modes/predict/#inference-arguments

In [22]:
def predict(path):
  results = model(path)
  result = results[0]
  boxes = result.boxes  # Boxes object for bounding box outputs
  masks = result.masks  # Masks object for segmentation masks outputs
  keypoints = result.keypoints  # Keypoints object for pose outputs
  probs = result.probs  # Probs object for classification outputs
  obb = result.obb  # Oriented boxes object for OBB outputs
  result.show()  # display to screen


  result.save(filename="result.jpg")  # save to disk
  return result.tojson(normalize=True)
  #return probs
#thing = predict("/content/drive/MyDrive/mewingAI/datasets/FacialFeatures_split/test/images/00a0fd8177a1db74_jpg.rf.e66cca8cc8cccf937b160ea6546b1fbb.jpg")

In [23]:
import numpy as np
import cv2

def preprocess_input(image_pil, net_h=416, net_w=416):
    image = np.asarray(image_pil)
    new_h, new_w, _ = image.shape
    # print("net:", net_h, net_w)
    # print("old:",new_h, new_w)
    # determine the new size of the image
    if (float(net_w)/new_w) < (float(net_h)/new_h):
        new_h = (new_h * net_w)/new_w
        new_w = net_w
    else:
        new_w = (new_w * net_h)/new_h
        new_h = net_h
    new_w = int(new_w)
    new_h = int(new_h)
    # print("new:",int(new_h), int(new_w))
    # resize the image to the new size
    #resized = cv2.resize(image[:,:,::-1]/255., (int(new_w), int(new_h)))
    resized = cv2.resize(image/255., (int(new_w), int(new_h)))

    # embed the image into the standard letter box
    # print("dims:",int((net_h-new_h)//2), int((net_h+new_h)//2), int((net_w-new_w)//2), int((net_w+new_w)//2))
    new_image = np.ones((net_h, net_w, 3)) * 0.5
    new_image[int((net_h-new_h)//2):int((net_h+new_h)//2), int((net_w-new_w)//2):int((net_w+new_w)//2), :] = resized
    new_image = np.expand_dims(new_image, 0)
    # print(new_image.shape)


    return new_image

Display image

In [54]:
def processImg(path):
  from PIL import Image
  from matplotlib import  pyplot as plt

  image_path = path
  image_pil = Image.open(image_path)
  image_w, image_h = image_pil.size
  new_image = preprocess_input(image_pil, 640, 640)

  new_image_pil = Image.fromarray((new_image[0] * 255).astype('uint8'))
  #new_image_pil.save("/content/new_img.jpg")

  #plt.imshow(new_image[0])
  #plt.show()

  return new_image_pil

In [25]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [26]:
import json

def readOutput(vals):
  jsonVals=json.loads(vals)
  names = []
  confidence=[]
  for item in jsonVals:
    #print(item)
    names.append(item['name'])
    confidence.append(item['confidence'])
  print(names)
  print(confidence)

  if ('openmouth' in names) and ('closedmouth' in names):
    openIndex=names.index('openmouth')
    closedIndex=names.index('closedmouth')
    if confidence[openIndex]>confidence[closedIndex]:
      return 'open'
    else:
      return 'closed'
  elif 'openmouth' in names:
    return 'open'
  else:
    return 'closed'

#result = readOutput(someJson)

NameError: name 'someJson' is not defined

In [27]:
def getResults(img):
  test = processImg(img)
  someJson = predict(test)
  result = readOutput(someJson)
  return result

#getResults(take_photo())

In [28]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo_auto(filename='photo.jpg', quality=0.8, delay=1):
    js = Javascript('''
    async function takePhoto(quality, delay) {
        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Wait for video to start, you can adjust the delay here
        await new Promise(resolve => setTimeout(resolve, delay));

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        video.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({}, {})'.format(quality, delay))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Example of calling the function
take_photo_auto(filename='photo.jpg', quality=0.8, delay=500)  # Adjust delay as needed


<IPython.core.display.Javascript object>

'photo.jpg'

In [52]:
from google.colab import output
#output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

#output.eval_js('new Audio("noise.mp3").play')

In [56]:
import time
from playsound import playsound

picture = take_photo_auto()

# Process the picture
results = getResults(picture)

def periodic_capture(interval, run_time):
  start_time = time.time()
  while (time.time() - start_time) < run_time:
    # Take a picture
    picture = take_photo_auto()

    # Process the picture
    results = getResults(picture)

    # Print or log the results
    if results == "open":
      #playsound('noise.mp3')
      output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
      #print("hi")

    # Wait for the next interval
    time.sleep(interval)

# Set the interval between captures (in seconds) and total run time (in seconds)
capture_interval = 2  # every 5 seconds
total_run_time = 60  # run for 60 seconds

periodic_capture(capture_interval, total_run_time)

<IPython.core.display.Javascript object>


0: 640x640 1 closedmouth, 1 face, 2 openeyes, 280.6ms
Speed: 2.6ms preprocess, 280.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth']
[0.83924, 0.56979, 0.48857, 0.48533]


<IPython.core.display.Javascript object>


0: 640x640 1 closedmouth, 1 face, 3 openeyes, 176.7ms
Speed: 2.2ms preprocess, 176.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth', 'openeye']
[0.85626, 0.59537, 0.50822, 0.44139, 0.25948]


<IPython.core.display.Javascript object>

...


0: 640x640 2 closedmouths, 1 face, 3 openeyes, 201.2ms
Speed: 2.2ms preprocess, 201.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openeye', 'openeye', 'closedmouth', 'closedmouth', 'openeye']
[0.84032, 0.54094, 0.51731, 0.50898, 0.36674, 0.27448]


<IPython.core.display.Javascript object>


0: 640x640 1 face, 2 openeyes, 1 openmouth, 170.8ms
Speed: 2.2ms preprocess, 170.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
['face', 'openmouth', 'openeye', 'openeye']
[0.88442, 0.64382, 0.60446, 0.50671]


<IPython.core.display.Javascript object>

KeyboardInterrupt: 